# Deploy ComfyUI on a RunPod GPU (3090)
This notebook builds and optionally runs `runpodctl` commands to favorite a template and create a comfy instance on a 3090 GPU.

**Assumptions:**
- `runpodctl` is installed and you are authenticated (run `runpodctl auth login`).
- Template identifiers and exact CLI subcommands may vary by `runpodctl` version. The notebook defaults to a dry run (safe).
- Update `DRY_RUN = False` to actually execute commands.

In [4]:
# Core imports and small helper runner
import subprocess, time, os, json
from pathlib import Path
DRY_RUN = True  # Set to False to actually run commands
def run(cmd):
    # prints the command and only executes if DRY_RUN is False
    print('\n$ ' + ' '.join(cmd))
    if not DRY_RUN:
        subprocess.run(cmd, check=True)

# Template history: stores recent template/gpu/deployment choices so the notebook can default to the last set
HISTORY_FILE = Path('notebooks/runpod_deployments/.templates_history.json')
MAX_HISTORY = 20

def load_history():
    if not HISTORY_FILE.exists():
        return []
    try:
        return json.loads(HISTORY_FILE.read_text()) or []
    except Exception as e:
        print('Failed to read history:', e)
        return []

def save_history(hist):
    try:
        HISTORY_FILE.parent.mkdir(parents=True, exist_ok=True)
        HISTORY_FILE.write_text(json.dumps(hist[:MAX_HISTORY], indent=2))
    except Exception as e:
        print('Failed to save history:', e)

def pick_template_interactive(history):
    """Return a dict with keys: template, gpu, deployment, extra"""
    default = history[0] if history else None
    if default:
        print('Default (last used):')
        print('  template:', default.get('template'))
        print('  gpu:', default.get('gpu'))
        print('  deployment:', default.get('deployment'))
    else:
        print('No template history found; you will be asked to provide values')

    # Offer choices from history
    if history:
        print('\nRecent templates:')
        for i, item in enumerate(history[:10], 1):
            print(f'  {i}. {item.get(
)} (gpu={item.get(
)}, deployment={item.get(
)})')
        print('  0. Enter a new template')
        choice = input('Choose number to reuse, or press Enter to accept default, or 0 to enter new: ').strip()
        if choice == '':
            return default
        if choice.isdigit():
            n = int(choice)
            if n == 0:
                pass  # fall through to manual entry
            elif 1 <= n <= len(history):
                return history[n-1]
    # Manual entry flow
    template = input('Enter template name (e.g. comfy/comfyui-template): ').strip()
    gpu = input('Enter gpu type (default 3090): ').strip() or '3090'
    deployment = input('Enter deployment name (leave blank for autogenerated): ').strip()
    if not deployment:
        deployment = f'comfy-{gpu}-{int(time.time())}'
    extra = input('Extra CLI args (optional): ').strip()
    return {'template': template, 'gpu': gpu, 'deployment': deployment, 'extra': extra}

# Load history and pick a configuration
history = load_history()
selected = pick_template_interactive(history)
template_name = selected.get('template') if isinstance(selected, dict) else selected
gpu_type = selected.get('gpu', '3090') if isinstance(selected, dict) else '3090'
deployment_name = selected.get('deployment', f'comfy-{gpu_type}-{int(time.time())}') if isinstance(selected, dict) else f'comfy-{gpu_type}-{int(time.time())}'
extra_args = selected.get('extra', '') if isinstance(selected, dict) else ''

# Persist selection to top of history
new_entry = {'template': template_name, 'gpu': gpu_type, 'deployment': deployment_name, 'extra': extra_args}
# remove identical entries
history = [e for e in history if e.get('template') != new_entry['template'] or e.get('gpu') != new_entry['gpu'] or e.get('deployment') != new_entry['deployment']]
history.insert(0, new_entry)
save_history(history)

print(f'Using template={template_name} gpu={gpu_type} deployment={deployment_name}')

SyntaxError: unterminated string literal (detected at line 46) (2085578933.py, line 46)

In [ ]:
#

## Favorite the ComfyUI template
Update `template_name` below to match your RunPod template identifier. If your `runpodctl` uses different subcommands, adjust accordingly.

In [ ]:
# Favorite the selected template (idempotent)
run(['runpodctl', 'templates', 'favorite', template_name])


$ runpodctl templates favorite comfy/comfyui-template


## Create the deployment
Adjust `gpu_type` and `deployment_name` as needed. The `--gpu` flag is used as a placeholder — adapt to your `runpodctl` version if it differs.

In [ ]:
deployment_name = 'comfy-3090-test'
gpu_type = '3090'
cmd = ['runpodctl', 'deployments', 'create', '--name', deployment_name, '--template', template_name, '--gpu', gpu_type]
run(cmd)

In [ ]:
# Optional: poll status (runs only when DRY_RUN is False)
if not DRY_RUN:
    for _ in range(30):
        subprocess.run(['runpodctl','deployments','status','--name',deployment_name])
        time.sleep(5)
else:
    print('Dry run complete — set DRY_RUN = False to execute')